In [1]:
using BenchmarkTools
using LinearAlgebra
using StaticArrays

# T(arg)


it generate(and return) a matrix of translation,the returned matrix have d+1 row and column where d is the lenght of the array passed as arg

In [3]:
function t(args...)
    d = length(args)
    mat = Matrix{Float64}(LinearAlgebra.I, d+1, d+1)
    for k in range(1, length=d)
            mat[k,d+1]=args[k]
    end
    return mat
end

t (generic function with 1 method)

In [3]:
@btime t(-0.5,-0.5)

  45.833 ns (1 allocation: 160 bytes)


3×3 Matrix{Float64}:
 1.0  0.0  -0.5
 0.0  1.0  -0.5
 0.0  0.0   1.0

In [4]:
@code_llvm t(-0.5,-0.5)

;  @ In[2]:1 within `t'
; Function Attrs: uwtable
define nonnull {}* @julia_t_1803(double %0, double %1) #0 {
pass:
;  @ In[2]:3 within `t'
; ┌ @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:471 within `Array'
   %2 = call nonnull {}* @j_Array_1805([1 x i8]* nocapture readonly @_j_const1, [2 x i64]* nocapture readonly @_j_const2) #0
; └
;  @ In[2]:5 within `t'
; ┌ @ array.jl within `setindex!'
   %3 = bitcast {}* %2 to {}**
   %4 = bitcast {}* %2 to double**
   %5 = getelementptr inbounds {}*, {}** %3, i64 3
   %6 = bitcast {}** %5 to i64*
; └
; ┌ @ array.jl:845 within `setindex!'
   %7 = load i64, i64* %6, align 8
   %.not = icmp eq i64 %7, 0
   br i1 %.not, label %oob, label %ib

ib:                                               ; preds = %pass
; └
; ┌ @ array.jl within `setindex!'
   %8 = getelementptr inbounds {}*, {}** %3, i64 4
   %9 = bitcast {}** %8 to i64*
; └
; ┌ @ array.jl:845 within `setindex!'
   %10 = load i64, i64

In [5]:
@benchmark t(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 985 evaluations.
 Range (min … max):  47.107 ns … 791.168 ns  ┊ GC (min … max): 0.00% … 77.35%
 Time  (median):     52.386 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   59.042 ns ±  39.261 ns  ┊ GC (mean ± σ):  3.13% ±  5.03%

  ▆██▆▄▅▄▃▃▂▁▁                                                 ▂
  █████████████▇█▆▆▅▅▅▆▅▅▆▅▅▅▆▅▆▆▆▆▅▅▄▅▁▅▄▅▅▄▆▆▆▆█▇▇▆▆▅▆▇██▆▆▅ █
  47.1 ns       Histogram: log(frequency) by time       170 ns <

 Memory estimate: 160 bytes, allocs estimate: 1.

# Improvment & changes

In [27]:
function t(args...)
    d = length(args)
    mat = MMatrix{d+1,d+1,Float64}(1I)
    @inbounds for k in range(1, length=d)
             mat[k,d+1]=args[k]
    end
    return mat
end

t (generic function with 1 method)

In [29]:
@btime t(-0.5,-0.5)

  13.914 ns (1 allocation: 80 bytes)


3×3 MMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 1.0  0.0  -0.5
 0.0  1.0  -0.5
 0.0  0.0   1.0

In [28]:
@benchmark t(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  13.814 ns …   3.410 μs  ┊ GC (min … max):  0.00% … 98.86%
 Time  (median):     14.715 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   21.615 ns ± 111.447 ns  ┊ GC (mean ± σ):  20.88% ±  4.06%

  █▇▅▃▃▂▁▁  ▂▂▁                                        ▁       ▂
  ██████████████▇▇▇▇▆▇▇▄▅▅▄▅▅▃▄▄▃▅▃▁▃▁▁▁▁▃▁▁▁▁▁▁▁▃▁▆▅█▇█▇▇▇▆▇▇ █
  13.8 ns       Histogram: log(frequency) by time      65.3 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

# Paralelization

In [2]:
function t(args...)
    d = length(args)
    mat = MMatrix{d+1,d+1,Float64}(1I)
    @inbounds @simd for k in range(1, length=d)
             mat[k,d+1]=args[k]
    end
    return mat
end

t (generic function with 1 method)

In [3]:
@benchmark t(-0.5,-0.5)

BenchmarkTools.Trial: 10000 samples with 996 evaluations.
 Range (min … max):  16.667 ns …   4.878 μs  ┊ GC (min … max):  0.00% … 97.07%
 Time  (median):     26.606 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   45.960 ns ± 132.715 ns  ┊ GC (mean ± σ):  10.93% ±  3.99%

  ▃▃▅█▃▂▂▃▂▅▅▂▁▂▁ ▁▁     ▁  ▁     ▁                            ▁
  ██████████████████▇██▇▆█▇▇█▇▇████▇▅▅▅▄▄▅▅▆▄▅▃▂▅▅▃▃▃▂▃▆▃▇▅█▆▅ █
  16.7 ns       Histogram: log(frequency) by time       181 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

## Just like the s funcion we have worse time with async and sync and an improvment with simd but only with the old funcion

In [2]:
function t(args...)
    d = length(args)
    mat = MMatrix{d+1,d+1,Float64}(1I)
    @async for k in range(1, length=d)
             mat[k,d+1]=args[k]
    end
    return mat
end

t (generic function with 1 method)

In [3]:
@benchmark t(-0.5,-0.5)

BenchmarkTools.Trial: 5071 samples with 759 evaluations.
 Range (min … max):  434.256 ns …  3.483 ms  ┊ GC (min … max):  0.00% … 99.98%
 Time  (median):     511.067 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.908 μs ± 55.325 μs  ┊ GC (mean ± σ):  70.90% ±  3.43%

    ▄█▅▁                                                        
  ▃▆████▆▅▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  434 ns          Histogram: frequency by time         1.51 μs <

 Memory estimate: 608 bytes, allocs estimate: 6.